In [2]:
import sys
sys.path.append("/home/ly/workspace/mmsa")
seed = 1938
import numpy as np
import torch
from torch import nn
from torch import optim

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
from models.lymodel5_no_gate import *
from utils.train import *
from typing import *
from utils.load_yelp import *
from utils.dataset import *
from utils.train import *
from utils.train import *

config

{'task': 'CLS',
 'embedding_dim': 100,
 'embedding': None,
 'freeze_embedding': True,
 'text_hidden_size': 100,
 'text_layers': 1,
 'max_tokens': 512,
 'uniform_bound': 0.1,
 'img_input_size': 2048,
 'img_encoder_layers': 1,
 'attention_nhead': 4,
 'fusion_nheads': 4,
 'dropout': 0.1,
 'output_size': 5,
 'bias_init': 1.0}

In [3]:
%%time
train_set, valid_set, test_set= load_glove_data(split811data, config)
batch_size = 64
workers = 4
train_loader, valid_loader, test_loader = get_loader(batch_size, workers, get_collate_fn(config), train_set, valid_set, test_set)
model = Model(config).cuda()
loss = nn.CrossEntropyLoss()

print(get_parameter_number(model), loss)

_interval = 5
lr = 1e-3
epoches = 50
stoping_step = 10
optimizer = get_regal_optimizer(model, optim.AdamW, lr)

viz = get_Visdom()
batch_loss_drawer = VisdomScalar(viz, f"batch_loss interval:{_interval}")
epoch_loss_drawer = VisdomScalar(viz, f"Train and valid loss", 2)
acc_drawer = VisdomScalar(viz, "Train and valid accuracy", 2)
text_writer = VisdomTextWriter(viz, "Training")

batch_loss = []
train_loss = []
valid_loss = []
train_acc = []
valid_acc = []

res, model = train_visdom_v2(model, optimizer, loss, viz, train_loader,
                          valid_loader, epoches, batch_loss, batch_loss_drawer,
                          train_loss, valid_loss, epoch_loss_drawer,
                          train_acc, valid_acc, acc_drawer, text_writer,
                         _interval=_interval, early_stop=stoping_step)

/home/ly/miniconda3/envs/torch1.7/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
Setting up a new session...


{'Total': 5137205, 'Trainable': 853805} CrossEntropyLoss()


No 1 epoch: 554it [01:15,  7.34it/s]
No 2 epoch: 554it [01:16,  7.28it/s]
No 3 epoch: 554it [01:16,  7.22it/s]
No 4 epoch: 554it [01:17,  7.15it/s]
No 5 epoch: 554it [01:17,  7.17it/s]
No 6 epoch: 554it [01:17,  7.12it/s]
No 7 epoch: 554it [01:16,  7.25it/s]
No 8 epoch: 554it [01:17,  7.17it/s]
No 9 epoch: 554it [01:16,  7.23it/s]
No 10 epoch: 554it [01:16,  7.22it/s]
No 11 epoch: 554it [01:17,  7.18it/s]
No 12 epoch: 554it [01:16,  7.20it/s]
No 13 epoch: 554it [01:17,  7.15it/s]
No 14 epoch: 554it [01:15,  7.31it/s]
No 15 epoch: 554it [01:17,  7.19it/s]
No 16 epoch: 554it [01:17,  7.13it/s]
No 17 epoch: 554it [01:17,  7.18it/s]
No 18 epoch: 554it [01:17,  7.17it/s]
No 19 epoch: 554it [01:17,  7.19it/s]
No 20 epoch: 554it [01:18,  7.06it/s]
No 21 epoch: 554it [01:18,  7.03it/s]
No 22 epoch: 554it [01:17,  7.11it/s]


CPU times: user 20min 20s, sys: 6min 21s, total: 26min 42s
Wall time: 31min 30s


In [4]:
res


{'min_valid_loss': 0.9483079119675853,
 'min_valid_loss_epoch': 12,
 'min_loss_train_acc': 0.6137960220059246,
 'min_valid_loss_train_loss': 0.8688888529928065,
 'min_loss_valid_acc': 0.5826185101580136,
 'last_valid_acc': 0.5656884875846501,
 'last_train_acc': 0.7493017350825222,
 'last_epoch': 22,
 'last_train_loss': 0.5964202659895901,
 'last_valid_loss': 1.3394649144489126}

In [5]:
eval_model(model, test_loader, loss)

/home/ly/miniconda3/envs/torch1.7/lib/python3.8/site-packages/torch/nn/modules/rnn.py:742: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729096996/work/aten/src/ATen/native/cudnn/RNN.cpp:775.)
  result = _VF.gru(input, batch_sizes, hx, self._flat_weights, self.bias,


((0.5796839729119638, 0.5844848811399804), 0.9311414723471648)